## PASO 0 EXTRACCION DE DATOS (df_v0)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
import mysql.connector

In [2]:
# REALIZAMOS LA EXTRACCION DE DATOS

# CARGAMOS EL CSV CON DATS NULL
df_Base0 = pd.read_csv('jamones_datos_step1.csv',sep =";" ,na_values = "NULL")

C:\Users\danny\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# SUSTITUIMOS 0 POR nan
df_Base1 = df_Base0.copy()
df_Base1.replace(0,np.nan,inplace=True)
df_Base1

,TipoArticuloEscandallo,Añada,Partida,Raza,Alimentacion,NumeroSerieLc,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,...,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,PesoEntradaEntrada,TipoPesada,PesoSalida,DescripcionArticulo,FechaEntrada,FechaSalida
0,Jamón,2014,102.0,IBERICO,BELLOTA,1.401001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"12,58547009",Grupal,"9,4",JAMÓN NO APTO PARA EL CONSUMO HUMANO,2014-01-13 00:00:00.000,2018-04-24 00:00:00.000
1,Jamón,2014,102.0,IBERICO,BELLOTA,1.401001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"12,58547009",Grupal,"10,4",JAMON DE BELLOTA IBERICO PASSION,2014-01-13 00:00:00.000,2018-06-04 00:00:00.000
2,Paleta,2014,102.0,IBERICO,BELLOTA,1.411001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"8,141025641",Grupal,"5,964",PALETA DE BELLOTA IBÉRICA D.O.,2014-01-13 00:00:00.000,2016-05-06 00:00:00.000
3,Paleta,2014,102.0,IBERICO,BELLOTA,1.411001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"8,141025641",Grupal,"3,15",PALETA DE BELLOTA IBÉRICA DESH.,2014-01-13 00:00:00.000,2016-12-01 00:00:00.000
4,Paleta,2014,102.0,IBERICO,BELLOTA,1.411001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"8,141025641",Grupal,"3,054",PALETA DE BELLOTA IB. D.O. DESH.,2014-01-13 00:00:00.000,2016-08-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158747,Paleta,2019,NaN,NaN,NaN,2.360144e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,"0,2",Grupal,"5,934",PALETA DE BELLOTA 100% IBÉRICA PASSION,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000
158748,Paleta,2019,NaN,NaN,NaN,2.360144e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,"0,2",Grupal,"6,06",PALETA DE BELLOTA 100% IBÉRICA PASSION,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000
158749,Paleta,2019,NaN,NaN,NaN,2.360182e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,"0,2",Grupal,"5,934",PALETA DE BELLOTA 100% IBÉRICA PASSION,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000
158750,Paleta,2019,NaN,NaN,NaN,2.360182e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,"0,2",Grupal,6,PALETA DE BELLOTA 100% IBÉRICA PASSION,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000


In [4]:
df_Base1.shape[0]

158752

In [5]:
df_Base1.isna().sum()

TipoArticuloEscandallo        0
Añada                         0
Partida                      24
Raza                        595
Alimentacion                 24
NumeroSerieLc                 2
AcidoOleico               93216
AcidoPalmitico            93216
AcidoEstearico            93216
AcidoLinoleico            93216
PesoVarios                    0
Pesovientres                  0
PesoCerdos                    0
PesoNetoCanal                 0
Nacimiento                29730
CodigoMatadero            16301
NroAnimales                7852
PesoEntradaEntrada            0
TipoPesada                    0
PesoSalida                 8299
DescripcionArticulo           9
FechaEntrada                  0
FechaSalida               10114
dtype: int64

## PASO 1 DATA ENGINEERING: MISSING DATA
    TipoArticuloEscandallo         0
    Añada                          0
    Partida                       24 <-- 1. Registros sin partida erróneos --> Eliminar
    Raza                         595 <-- 2. Registros se clasifican por alimentación y raza --> Eliminar
    Alimentacion                  24 <-- 3. Registros se clasifican por alimentación y raza --> Eliminar
    NumeroSerieLc                  2 <-- 4. Eliminar
    AcidoOleico               118882 <-- Mantener
    AcidoPalmitico            118882 <-- Mantener
    AcidoEstearico            118882 <-- Mantener
    AcidoLinoleico            118882 <-- Mantener
    PesoVarios                 25458 <-- Mantener 
    Pesovientres               26552 <-- Mantener 
    PesoCerdos                 26552 <-- Mantener 
    PesoNetoCanal              26552 <-- Mantener 
    Nacimiento                 29730 <-- Número alto de registros, si solo falta nacimiento --> Mantener
    CodigoMatadero             16301 <-- No tiene mucha relevancia --> Mantener
    NroAnimales                 7852 <-- Hay partidas únicamente de jamones y unicamente de paletas: no relevante --> Mantener
    PesoEntradaEntrada             0
    TipoPesada                     0
    PesoSalida                 19426 <-- Registros de piezas que aún no han salido --> Mantener
    DescripcionArticulo            9 <-- No relevante --> Mantener
    FechaEntrada                   0
    FechaSalida                10114 <-- Registros de piezas que aún no han salido --> Mantener
    dtype: int64

In [6]:
# PASO 1 MISSING DATA 
# Partida                       24 <-- 1. Registros sin partida erróneos --> Eliminar
df_MssDt1 = df_Base1.copy()
df_MssDt1 = df_MssDt1.drop(df_MssDt1[df_MssDt1['Partida'].isna()].index)
df_MssDt1.shape[0]


158728

In [7]:
# PASO 2 MISSING DATA 
# Raza                         595 <-- 2. Registros se clasifican por alimentación y raza --> Eliminar

df_MssDt2 = df_MssDt1.copy()
df_MssDt2 = df_MssDt2.drop(df_MssDt2[df_MssDt2['Raza'].isna()].index)
df_MssDt2.shape[0]

158157

In [8]:
# PASO 3 MISSING DATA 
# Alimentacion                  24 <-- 3. Registros se clasifican por alimentación y raza --> Eliminar

df_MssDt3 = df_MssDt2.copy()
df_MssDt3 = df_MssDt3.drop(df_MssDt3[df_MssDt3['Alimentacion'].isna()].index)
df_MssDt3.shape[0]

158157

In [9]:
# PASO 4 MISSING DATA 
# NumeroSerieLc                  2 <-- 4. Eliminar

df_MssDt4 = df_MssDt3.copy()
df_MssDt4 = df_MssDt4.drop(df_MssDt4[df_MssDt4['NumeroSerieLc'].isna()].index)
df_MssDt4.shape[0]

158155

## PASO 2 DATA ENGINEERING: WRONG DATA

    TipoArticuloEscandallo         2 
    Añada                          9
    Partida                      524 
    Raza                           5 <-- 1. Hay sólo 3 razas, corregir clasificación
    Alimentacion                   3
    NumeroSerieLc             129457 <-- 2. Hay muchos duplicados --> eliminar --> no sirve como ID para tabla pieza en la BBDD.
    AcidoOleico                  107
    AcidoPalmitico               110
    AcidoEstearico               104
    AcidoLinoleico               107
    PesoVarios                   428 
    Pesovientres                 423
    PesoCerdos                   391
    PesoNetoCanal                423
    Nacimiento                   207 <-- 3. Hay entradas incorrectar no útiles: borrar información de columna incorrecta
    CodigoMatadero                 4
    NroAnimales                  145
    PesoEntradaEntrada          2178 <-- 4. Todos los pesos grupales son Null ya que entregan información falsa
    TipoPesada                     2
    PesoSalida                  3355 <-- 5. Identificar pesadas que no sean de la pieza entera (p.ej. deshuesadas)
    DescripcionArticulo          440 <-- 6. Simplificar categorías
    FechaEntrada                 298 <-- 7. Checkear que la diferencia entre fecha de entrada y salida sea coherente
    FechaSalida                 1697
    dtype: int64

  

    
    

In [10]:
# PASO 1 WRONG DATA
# Raza                           5 <-- 1. Hay sólo 3 razas, corregir clasificación

df_WngDt1 = df_MssDt4.copy()
df_WngDt1['Raza'].replace(to_replace={'IBERICO': '50% IBERICO', 'IBERICO PURO': '100% IBERICO'}, inplace=True)
df_WngDt1['Raza'].unique()


array(['50% IBERICO', '75% IBERICO', '100% IBERICO'], dtype=object)

In [11]:
# PASO 2 WRONG DATA
# NumeroSerieLc             129457 <-- 2. Hay muchos duplicados --> eliminar --> no sirve como ID para tabla pieza en la BBDD.
df_WngDt2=df_WngDt1.copy()
print(df_WngDt2.shape[0])
print(df_WngDt2["NumeroSerieLc"].unique().shape[0])
print(df_WngDt2["NumeroSerieLc"].nunique())

158155
128892
128892


In [12]:
# PASO 2 WRONG DATA
# NumeroSerieLc             129457 <-- 2. Hay muchos duplicados --> eliminar --> no sirve como ID para tabla pieza en la BBDD.

df_WngDt2.drop_duplicates(inplace=True)
df_WngDt2.shape[0]

150433

In [13]:
# PASO 3 WRONG DATA
# Nacimiento                   207 <-- 3. Hay entradas incorrectar no útiles: borrar información de columna incorrecta
df_WngDt3=df_WngDt2.copy()
df_WngDt3 = df_WngDt3.drop(df_WngDt3[df_WngDt3['Nacimiento'].str.len()!=10].index)
df_WngDt3.shape[0]

120481

In [14]:
df_WngDt2[df_WngDt2['Nacimiento'].str.len()!=10]

,TipoArticuloEscandallo,Añada,Partida,Raza,Alimentacion,NumeroSerieLc,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,...,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,PesoEntradaEntrada,TipoPesada,PesoSalida,DescripcionArticulo,FechaEntrada,FechaSalida
0,Jamón,2014,102.0,50% IBERICO,BELLOTA,1.401001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"12,58547009",Grupal,"9,4",JAMÓN NO APTO PARA EL CONSUMO HUMANO,2014-01-13 00:00:00.000,2018-04-24 00:00:00.000
1,Jamón,2014,102.0,50% IBERICO,BELLOTA,1.401001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"12,58547009",Grupal,"10,4",JAMON DE BELLOTA IBERICO PASSION,2014-01-13 00:00:00.000,2018-06-04 00:00:00.000
2,Paleta,2014,102.0,50% IBERICO,BELLOTA,1.411001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"8,141025641",Grupal,"5,964",PALETA DE BELLOTA IBÉRICA D.O.,2014-01-13 00:00:00.000,2016-05-06 00:00:00.000
3,Paleta,2014,102.0,50% IBERICO,BELLOTA,1.411001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"8,141025641",Grupal,"3,15",PALETA DE BELLOTA IBÉRICA DESH.,2014-01-13 00:00:00.000,2016-12-01 00:00:00.000
4,Paleta,2014,102.0,50% IBERICO,BELLOTA,1.411001e+11,"53,76","21,02","10,61","8,11",...,"16351,398",NaN,1.0,117.0,"8,141025641",Grupal,"3,054",PALETA DE BELLOTA IB. D.O. DESH.,2014-01-13 00:00:00.000,2016-08-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156203,Paleta,2016,9366.0,50% IBERICO,CEBO,2.571002e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,29.0,"7,8",Grupal,"2,682",PALETA DE CEBO IBÉRICA DESH,2016-11-04 00:00:00.000,2018-07-02 00:00:00.000
156204,Paleta,2016,9366.0,50% IBERICO,CEBO,2.571002e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,29.0,"7,8",Grupal,"2,682",PALETA DE CEBO IBÉRICA DESH,2016-11-04 00:00:00.000,2018-07-16 00:00:00.000
156205,Paleta,2016,9366.0,50% IBERICO,CEBO,2.571002e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,29.0,"7,8",Grupal,"4,785",PALETA DE CEBO IBÉRICA,2016-11-04 00:00:00.000,2018-10-04 00:00:00.000
156206,Paleta,2016,9366.0,50% IBERICO,CEBO,2.571002e+11,NaN,NaN,NaN,NaN,...,0,NaN,NaN,29.0,"7,8",Grupal,"4,721",PALETA DE CEBO IBÉRICA,2016-11-04 00:00:00.000,2018-10-04 00:00:00.000


In [15]:
# PASO 4 WRONG DATA
# PesoEntradaEntrada          2178 <-- 4. Hay que chequear que los pesos sean coherentes

# Los pesos de entrada grupales son una media obtenida del peso total de la partida y el número de especimenes.
# Los eliminaremos ya que los valores son falsos y se podrían volver a calcular.

df_WngDt4 = df_WngDt3.copy()

df_WngDt4["PesoEntradaEntrada"] = np.where (df_WngDt4['TipoPesada']=="Grupal", np.nan, df_WngDt4["PesoEntradaEntrada"])

df_WngDt4[df_WngDt4['TipoPesada']=="Grupal"]


,TipoArticuloEscandallo,Añada,Partida,Raza,Alimentacion,NumeroSerieLc,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,...,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,PesoEntradaEntrada,TipoPesada,PesoSalida,DescripcionArticulo,FechaEntrada,FechaSalida
480,Jamón,2014,1025.0,50% IBERICO,BELLOTA,1.401210e+11,"57,4","18,99","8,23","8,85",...,"17295,824",20/05/2012,1.0,130.0,NaN,Grupal,"8,755",JAMON DE BELLOTA IBERICO PASSION,2014-03-20 00:00:00.000,2018-06-04 00:00:00.000
481,Jamón,2014,1025.0,50% IBERICO,BELLOTA,1.401210e+11,"57,4","18,99","8,23","8,85",...,"17295,824",20/05/2012,1.0,130.0,NaN,Grupal,"7,755","JAMON BELLOTA IBERICO 75% RAZA IBERICA75% 7,0-...",2014-03-20 00:00:00.000,2018-10-31 00:00:00.000
482,Jamón,2014,1025.0,50% IBERICO,BELLOTA,1.401210e+11,"57,4","18,99","8,23","8,85",...,"17295,824",20/05/2012,1.0,130.0,NaN,Grupal,"7,755","JAMON BELLOTA IBERICO 75% RAZA IBERICA75% 7,0-...",2014-03-20 00:00:00.000,2018-10-31 00:00:00.000
483,Jamón,2014,1025.0,50% IBERICO,BELLOTA,1.401210e+11,"57,4","18,99","8,23","8,85",...,"17295,824",20/05/2012,1.0,130.0,NaN,Grupal,"7,4",JAMON IBERICO BELLOTA PIEZA UN DONIBERIIBERICO...,2014-03-20 00:00:00.000,2017-12-01 00:00:00.000
484,Paleta,2014,1025.0,50% IBERICO,BELLOTA,1.411210e+11,"57,4","18,99","8,23","8,85",...,"17295,824",20/05/2012,1.0,130.0,NaN,Grupal,"6,64",PALETA DE BELLOTA IBÉRICA PASSION,2014-03-20 00:00:00.000,2017-08-28 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158723,Paleta,2018,999.0,75% IBERICO,BELLOTA,2.370306e+11,NaN,NaN,NaN,NaN,...,"9649,962",10/05/2016,1.0,65.0,NaN,Grupal,"6,383",PALETA DE BELLOTA IBÉRICA,2018-02-12 00:00:00.000,2021-04-13 00:00:00.000
158724,Paleta,2018,999.0,75% IBERICO,BELLOTA,2.370306e+11,NaN,NaN,NaN,NaN,...,"9649,962",10/05/2016,1.0,65.0,NaN,Grupal,"3,16",PALETA DE BELLOTA IBÉRICA,2018-02-12 00:00:00.000,2021-04-13 00:00:00.000
158725,Paleta,2018,999.0,75% IBERICO,BELLOTA,2.370306e+11,NaN,NaN,NaN,NaN,...,"9649,962",10/05/2016,1.0,65.0,NaN,Grupal,"6,383",PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12 00:00:00.000,2021-04-16 00:00:00.000
158726,Paleta,2018,999.0,75% IBERICO,BELLOTA,2.370306e+11,NaN,NaN,NaN,NaN,...,"9649,962",10/05/2016,1.0,65.0,NaN,Grupal,"3,16",PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12 00:00:00.000,2021-04-16 00:00:00.000


In [16]:
# PASO 5 WRONG DATA
# PesoSalida                  3355 <-- 5. Identificar pesadas que no sean de la pieza entera (p.ej. deshuesadas)

import re

# Hay pesadas deshuesadas, en blisters y de pieza entera
# Para la BBDD haremos una clasificación previa en funcion del tipo de pesada ya que a posteriori solo se usarán las pesadas
# de pieza entera.


df_WngDt5 = df_WngDt4.copy()

df_WngDt5["TipoPesadaSalida"]=""

df_WngDt5["TipoPesadaSalida"] = np.where (df_WngDt5["DescripcionArticulo"].str.contains("DESH"), "Pesada pieza deshuesada", df_WngDt5["TipoPesadaSalida"])
df_WngDt5["TipoPesadaSalida"] = np.where (df_WngDt5["DescripcionArticulo"].str.contains("CAJA")|df_WngDt5['DescripcionArticulo'].str.contains("MADERA"), "Pesada blisters para caja",df_WngDt5["TipoPesadaSalida"])
df_WngDt5["TipoPesadaSalida"] = np.where (df_WngDt5["TipoPesadaSalida"]=="", "Pesada PIEZA ENTERA", df_WngDt5["TipoPesadaSalida"])

df_WngDt5["EstadoProductoSalida"] = np.where (df_WngDt5['DescripcionArticulo'].str.contains("APT"), "NO APTO PARA CONSUMO", "APTO PARA CONSUMO")


df_WngDt5["TipoPesadaSalida"].unique()

array(['Pesada PIEZA ENTERA', 'Pesada pieza deshuesada',
       'Pesada blisters para caja'], dtype=object)

In [17]:
# PASO 6 WRONG DATA
# FechaEntrada                 298 <-- 6. Checkear que la diferencia entre fecha de entrada y salida sea coherente.

from datetime import datetime as dt

df_WngDt6 = df_WngDt5.copy()

df_WngDt6['FechaEntrada'] =  pd.to_datetime(df_WngDt6['FechaEntrada'])
df_WngDt6['FechaSalida'] =  pd.to_datetime(df_WngDt6['FechaSalida'])

# Los jamones que tengan menos de 15 meses en almacen se guardarán con una categoría en EstadoProductoSalida 
# como SALIDO PREMATURA para separarlos para el posterior estudio

df_WngDt6["EstadoProductoSalida"]=np.where((round((df_WngDt6["FechaSalida"]-df_WngDt6["FechaEntrada"]).dt.days)/30)<15, "Salida Prematura", "Salida Correcta")
df_WngDt6["EstadoProductoSalida"]

480       Salida Correcta
481       Salida Correcta
482       Salida Correcta
483       Salida Correcta
484       Salida Correcta
               ...       
158723    Salida Correcta
158724    Salida Correcta
158725    Salida Correcta
158726    Salida Correcta
158727    Salida Correcta
Name: EstadoProductoSalida, Length: 120481, dtype: object

In [18]:
df_WngDt6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120481 entries, 480 to 158727
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   TipoArticuloEscandallo  120481 non-null  object        
 1   Añada                   120481 non-null  int64         
 2   Partida                 120481 non-null  float64       
 3   Raza                    120481 non-null  object        
 4   Alimentacion            120481 non-null  object        
 5   NumeroSerieLc           120481 non-null  float64       
 6   AcidoOleico             54536 non-null   object        
 7   AcidoPalmitico          54536 non-null   object        
 8   AcidoEstearico          54536 non-null   object        
 9   AcidoLinoleico          54536 non-null   object        
 10  PesoVarios              120481 non-null  object        
 11  Pesovientres            120481 non-null  object        
 12  PesoCerdos              1204

## PASO 3 DATA ENGINEERING: DATA TYPING
    
    <class 'pandas.core.frame.DataFrame'>
    Int64Index: 148720 entries, 0 to 158727
    Data columns (total 25 columns):
     #   Column                  Non-Null Count   Dtype         
    ---  ------                  --------------   -----         
     0   TipoArticuloEscandallo  148720 non-null  object        
     1   Añada                   148720 non-null  int64         
     2   Partida                 148720 non-null  float64 <-- Convertir a tipo int64         
     3   Raza                    148720 non-null  object        
     4   Alimentacion            148720 non-null  object        
     5   NumeroSerieLc           148720 non-null  float64 <-- Convertir a tipo int64       
     6   AcidoOleico             38025 non-null   float64       
     7   AcidoPalmitico          38025 non-null   float64       
     8   AcidoEstearico          38025 non-null   float64       
     9   AcidoLinoleico          38025 non-null   float64       
     10  PesoVarios              124682 non-null  float64       
     11  Pesovientres            123596 non-null  float64       
     12  PesoCerdos              123596 non-null  float64       
     13  PesoNetoCanal           123596 non-null  float64       
     14  Nacimiento              120762 non-null  object  <-- Convertir a tipo Date        
     15  CodigoMatadero          133264 non-null  float64 <-- Convertir a tipo int64       
     16  NroAnimales             141486 non-null  float64 <-- Convertir a tipo int64       
     17  PesoEntradaEntrada      10150 non-null   float64       
     18  TipoPesada              148720 non-null  object        
     19  PesoSalida              130136 non-null  float64       
     20  DescripcionArticulo     148711 non-null  object        
     21  FechaEntrada            148720 non-null  datetime64[ns]
     22  FechaSalida             138882 non-null  datetime64[ns]
     23  TipoPesadaSalida        148720 non-null  object        
     24  EstadoProductoSalida    148720 non-null  object        
    dtypes: datetime64[ns](2), float64(14), int64(1), object(8)
    memory usage: 29.5+ MB   
    

In [20]:
df_DtTyp = df_WngDt6.copy()

df_DtTyp["CodigoMatadero"] = df_DtTyp["CodigoMatadero"].fillna(-1) 
df_DtTyp["NroAnimales"] = df_DtTyp["NroAnimales"].fillna(-1) 


df_DtTyp["Partida"] = df_DtTyp["Partida"].astype("int64")
df_DtTyp["NumeroSerieLc"] = df_DtTyp["NumeroSerieLc"].astype("int64")
df_DtTyp['Nacimiento'] =  pd.to_datetime(df_DtTyp['Nacimiento'])
df_DtTyp["CodigoMatadero"] = df_DtTyp["CodigoMatadero"].astype("int64")
df_DtTyp["NroAnimales"] = df_DtTyp["NroAnimales"].astype("int64")
df_DtTyp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120481 entries, 480 to 158727
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   TipoArticuloEscandallo  120481 non-null  object        
 1   Añada                   120481 non-null  int64         
 2   Partida                 120481 non-null  int64         
 3   Raza                    120481 non-null  object        
 4   Alimentacion            120481 non-null  object        
 5   NumeroSerieLc           120481 non-null  int64         
 6   AcidoOleico             54536 non-null   object        
 7   AcidoPalmitico          54536 non-null   object        
 8   AcidoEstearico          54536 non-null   object        
 9   AcidoLinoleico          54536 non-null   object        
 10  PesoVarios              120481 non-null  object        
 11  Pesovientres            120481 non-null  object        
 12  PesoCerdos              1204

In [22]:
df_DtTyp.to_csv('Step1JamonesDataExtraction.csv',sep=";",index=False)